<a href="https://colab.research.google.com/github/akshar27/CNN-flowers-pytorch/blob/master/Cnn_using_pytorch_flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
transform = transforms.Compose([
    transforms.RandomCrop((224, 224)),  # Randomly crop the image
    transforms.ToTensor()               # Convert the image to a tensor
])

In [3]:
train_data = datasets.Flowers102(root='/flowers', transform=transform, split="train", download=True)

100%|██████████| 344862509/344862509 [00:12<00:00, 27437501.22it/s]


Extracting /flowers/flowers-102/102flowers.tgz to /flowers/flowers-102


100%|██████████| 502/502 [00:00<00:00, 414313.38it/s]


100%|██████████| 14989/14989 [00:00<00:00, 13160649.50it/s]


In [4]:
test_data = datasets.Flowers102(root='/flowers', transform=transform, split="test", download=True)

In [5]:
train_data

Dataset Flowers102
    Number of datapoints: 1020
    Root location: /flowers
    split=train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(224, 224), padding=None)
               ToTensor()
           )

In [6]:
test_data

Dataset Flowers102
    Number of datapoints: 6149
    Root location: /flowers
    split=test
    StandardTransform
Transform: Compose(
               RandomCrop(size=(224, 224), padding=None)
               ToTensor()
           )

In [7]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)

In [8]:
for i, (X_train, X_test) in enumerate(train_data):
  break

In [9]:
X_train.shape

torch.Size([3, 224, 224])

In [10]:
X_test

0

In [11]:
conv1 = nn.Conv2d(3, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3 ,1)

In [12]:
x = X_train.view(1, 3, 224, 224)

In [13]:
x = F.relu(conv1(x))

In [14]:
x.shape

torch.Size([1, 6, 222, 222])

In [15]:
x = F.max_pool2d(x, 2,2)

In [16]:
x.shape

torch.Size([1, 6, 111, 111])

In [17]:
x = F.relu(conv2(x))
x = F.max_pool2d(x, 2,2)
x.shape

torch.Size([1, 16, 54, 54])

In [18]:
cd/

/


In [19]:
ls

bin@                        datalab/  home/    lib64@   NGC-DL-CONTAINER-LICENSE  run/   tmp/
boot/                       dev/      kaggle/  libx32@  opt/                      sbin@  tools/
content/                    etc/      lib@     media/   proc/                     srv/   usr/
cuda-keyring_1.0-1_all.deb  flowers/  lib32@   mnt/     root/                     sys/   var/


In [20]:
cd flowers

/flowers


In [21]:
ls

flowers-102/


In [22]:
cd flowers-102

/flowers/flowers-102


In [23]:
ls


102flowers.tgz  imagelabels.mat  jpg/  setid.mat


In [28]:
class FlowerCNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)

    self.fc1 = nn.Linear(54*54*16, 400)
    # self.fc2 = nn.Linear(10000, 400)
    self.fc3 = nn.Linear(400, 120)
    # self.fc4 = nn.Linear(84, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2, 2)

    x = x.view(-1, 54*54*16)
    x = F.relu(self.fc1(x))
    # x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    # x = F.relu(self.fc4(x))
    return F.log_softmax(x, dim=1)





In [29]:
model = FlowerCNN()
model

FlowerCNN(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=46656, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=120, bias=True)
)

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [34]:
import time
start_time = time.time()

epochs =5
train_loss =[]
test_loss =[]
train_correct = []
test_correct = []

for i in range(epochs):
  trn_crr = 0
  tst_crr =0

  for b, (X_train, y_train) in enumerate(train_loader):
    b+=1
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    predicted = torch.max(y_pred.data, 1)[1]
    batch_crr = (predicted==y_train).sum(0)
    trn_crr += batch_crr

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if b%600 == 0:
      print(f'Epoch: {i}, Batch: {b}, Loss: {loss.item()}')

  train_loss.append(loss)
  train_correct.append(trn_crr)


  with torch.no_grad():
    for b, (X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1]
      tst_crr += (predicted==y_test).sum(0)

    loss = criterion(y_val, y_test)
    test_loss.append(loss)
    test_correct.append(tst_crr)



current_time = time.time()
total_time = current_time - start_time
print(f'Time : {total_time/60}')

Time : 10.53891932964325
